<a href="https://colab.research.google.com/github/Austfi/SNOWPACKforPatrollers/blob/dev/SNOWPACKforPatrollers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SNOWPACK Model Setup and Execution

This notebook guides you through the process of setting up and running the SNOWPACK model. It includes steps for installing necessary libraries, compiling the SNOWPACK and MeteoIO code, configuring the model, fetching meteorological data from historical weather models, and running SNOWPACK.

# The following cells below set up SNOWPACK AND MeteoIO and the PATH structure to run it. They should not be edited. Pressing the play button below will run them all.

In [34]:
# @title Install environment updates
!apt-get update
!apt-get install -y build-essential cmake git liblapack-dev numdiff

%pip install openmeteo-requests requests-cache retry-requests pandas numpy

import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import os
from pathlib import Path
import openmeteo_requests
import requests_cache
from retry_requests import retry
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import json
from typing import Optional



Hit:1 https://cli.github.com/packages stable InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,594 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [3,855 kB]
Fetched 5,833 kB in 2s (3,699 kB

In [35]:
# @title Download SNOWPACK
!git clone https://github.com/snowpack-model/snowpack.git


fatal: destination path 'snowpack' already exists and is not an empty directory.


In [36]:
# @title Make  ~/usr folder and remove sample data
!mkdir ~/usr/
!rm -rf /content/sample_data

mkdir: cannot create directory ‘/root/usr/’: File exists


In [37]:
# @title Compile MeteoIO  (Runs about 6 Mins)
!cd /content/snowpack/Source/meteoio/ && \
cmake -S . -B build-meteoio -DCMAKE_BUILD_TYPE=Release -DCMAKE_INSTALL_PREFIX=$HOME/usr && \
cmake --build build-meteoio -j2 && \
cmake --install build-meteoio

-- Could NOT find Doxygen (missing: DOXYGEN_EXECUTABLE) 
-- Could NOT find Doxygen (missing: DOXYGEN_EXECUTABLE) 
-- Configuring done (0.2s)
-- Generating done (0.1s)
-- Build files have been written to: /content/snowpack/Source/meteoio/build-meteoio
[  0%] Building C object meteoio/CMakeFiles/meteoio.dir/thirdParty/tinyexpr.c.o
[  1%] Building C object meteoio/CMakeFiles/meteoio.dir/thirdParty/ctsa/autoutils.c.o
[  1%] Building C object meteoio/CMakeFiles/meteoio.dir/thirdParty/ctsa/boxcox.c.o
[  2%] Building C object meteoio/CMakeFiles/meteoio.dir/thirdParty/ctsa/boxjenkins.c.o
[  2%] Building C object meteoio/CMakeFiles/meteoio.dir/thirdParty/ctsa/brent.c.o
[  3%] Building C object meteoio/CMakeFiles/meteoio.dir/thirdParty/ctsa/conjgrad.c.o
[  3%] Building C object meteoio/CMakeFiles/meteoio.dir/thirdParty/ctsa/conv.c.o
[  4%] Building C object meteoio/CMakeFiles/meteoio.dir/thirdParty/ctsa/ctsa.c.o
[  4%] Building C object meteoio/CMakeFiles/meteoio.dir/thirdParty/ctsa/dist.c.o
[  

In [38]:
# @title Compile SNOWPACK (Runs about 2 mins)

!cd /content/snowpack/Source/snowpack/ && \
cmake -S . -B build-snowpack -DCMAKE_BUILD_TYPE=Release \
      -DCMAKE_PREFIX_PATH=$HOME/usr \
      -DDEBUG_ARITHM=OFF \
      -DENABLE_LAPACK=ON \
      -DMETEOIO_LIBRARY=$HOME/usr/lib/libmeteoio.so && \
cmake --build build-snowpack -j2 && \
cmake --install build-snowpack

-- Found METEOIO 
-- Found Lapack 
-- Could NOT find Doxygen (missing: DOXYGEN_EXECUTABLE) 
-- Could NOT find Doxygen (missing: DOXYGEN_EXECUTABLE) 
-- Configuring done (0.1s)
-- Generating done (0.0s)
-- Build files have been written to: /content/snowpack/Source/snowpack/build-snowpack
[  6%] Building CXX object snowpack/CMakeFiles/snowpack.dir/DataClasses.cc.o
[  6%] Building CXX object snowpack/CMakeFiles/snowpack.dir/vanGenuchten.cc.o
[  9%] Building CXX object snowpack/CMakeFiles/snowpack.dir/SnowpackConfig.cc.o
[ 12%] Building CXX object snowpack/CMakeFiles/snowpack.dir/Meteo.cc.o
[ 16%] Building CXX object snowpack/CMakeFiles/snowpack.dir/Saltation.cc.o
gmake[2]: *** [snowpack/CMakeFiles/snowpack.dir/build.make:121: snowpack/CMakeFiles/snowpack.dir/Meteo.cc.o] Interrupt
gmake[2]: *** [snowpack/CMakeFiles/snowpack.dir/build.make:135: snowpack/CMakeFiles/snowpack.dir/Saltation.cc.o] Interrupt
gmake[1]: *** [CMakeFiles/Makefile2:195: snowpack/CMakeFiles/snowpack.dir/all] Interrupt


In [ ]:
# @title Set up PATH to compiled code
import os

# Set PATH and LD_LIBRARY_PATH in Python environment
# Note: These paths are for Colab. In local environments, snowpack is typically in /usr/local/bin
os.environ["PATH"] = "/home/app/usr/bin:" + os.environ.get("PATH", "")
os.environ["LD_LIBRARY_PATH"] = "/home/app/usr/lib:" + os.environ.get("LD_LIBRARY_PATH", "")

# Verify snowpack is accessible
!which snowpack

# The above cells have set up SNOWPACK and made it ready to run. The below cells create the files, collect data, and run the snowpack model. This generates snowprofiles with no snow to start the model from.

In [ ]:
#@title SNOWPACK Configuration Files Generation
"""
Generate .sno and .ini configuration files for SNOWPACK simulations
"""

#@markdown ## Station Configuration
station_id = "watrous"  #@param {"type":"string"}
station_name = "watrous_E_NTL"  #@param {"type":"string"}
latitude = 39.71438  #@param {"type":"number"}
longitude = -105.84475  #@param {"type":"number"}
altitude = 11800  #@param {"type":"integer"}
altitude_unit = "feet"  # @param ["meters", "feet"]
timezone = -7  #@param {"type":"number"}
profile_date = "2024-11-01T00:00:00"  #@param {"type":"string"}
coord_sys = "UTM" #@param {"type":"string"}
coord_param = "13S" #@param {"type":"string"}

#@markdown ## Virtual Slopes Configuration
num_slopes = 5  #@param {"type":"number", "min":1, "max":10}
include_flat = True  #@param {"type":"boolean"}
default_slope_angle = 38.0  #@param {"type":"number", "min":0, "max":90}

#@markdown ## Slope Directions (degrees: 0=North, 90=East, 180=South, 270=West)
north_slope = True  #@param {"type":"boolean"}
east_slope = True  #@param {"type":"boolean"}
south_slope = True  #@param {"type":"boolean"}
west_slope = True  #@param {"type":"boolean"}
custom_directions = ""  #@param {"type":"string"} {description:"Comma-separated azimuth angles (e.g., 45,135,225,315)"}

#@markdown ## SNOWPACK Settings
meas_tss = "false"  #@param ["true","false"]
enforce_measured_snow_heights = "false"  #@param ["true","false"]
write_profiles = "true"  #@param ["true","false"]
write_timeseries = "false"  #@param ["true","false"]
write_snowpack = "false"  #@param ["true","false"]
snow_erosion = "redeposit"  #@param ["true", "redeposit", "free", "virtual", "none"]
snow_redistribution = "advanced"  #@param ["true", "simple", "advanced", "none"]
hs_density_parameterization = "LEHNING_NEW"  #@param ["LEHNING_OLD", "LEHNING_NEW", "BELLAIRE", "ZWART", "JORDY", "PAHAUT", "NIED", "VANKAMPENHOUT", "KRAMPE"]

#@markdown ## Output Configuration
sno_directory = "/content/input"  #@param {"type":"string"}
ini_directory = "/content/config"  #@param {"type":"string"}
generate_config_files = True  #@param {"type":"boolean"}

# @markdown ### SNOWPACK Run End Date
snowpack_end_date_input = "2025-04-01"  # @param {type:"date"}
snowpack_end_date = f"{snowpack_end_date_input}T00:00"

# For local testing - use current directory
import os
if not os.path.exists("/content"):
    sno_directory = "."
    ini_directory = "."

def generate_slopes(include_flat, north_slope, east_slope, south_slope, west_slope, custom_directions, default_slope_angle):
    """Generate list of virtual slopes based on user selections"""
    slopes = []

    # Add flat slope if requested
    if include_flat:
        slopes.append((0.0, 0.0))

    # Add cardinal direction slopes
    if north_slope:
        slopes.append((default_slope_angle, 0.0))
    if east_slope:
        slopes.append((default_slope_angle, 90.0))
    if south_slope:
        slopes.append((default_slope_angle, 180.0))
    if west_slope:
        slopes.append((default_slope_angle, 270.0))

    # Add custom directions
    if custom_directions.strip():
        try:
            custom_angles = [float(x.strip()) for x in custom_directions.split(',')]
            for angle in custom_angles:
                if 0 <= angle <= 360:
                    slopes.append((default_slope_angle, angle))
        except ValueError:
            print("Warning: Invalid custom directions format. Use comma-separated numbers.")

    return slopes

def to_meters(value: float, unit: str) -> float:
    """Normalize altitude inputs so downstream code always works in meters."""
    unit = unit.lower()
    if unit == "meters":
        return float(value)
    if unit == "feet":
        return float(value) * 0.3048
    raise ValueError(f"Unsupported altitude unit: {unit}")

# Using variables from the configuration cell
altitude_meters = to_meters(altitude, altitude_unit)

def create_sno_content(station_id, station_name, longitude, latitude, altitude_meters, timezone, profile_date, slope_angle, slope_azimuth):
    """Create .sno file content for a single slope"""

    content = f"""SMET 1.1 ASCII
[HEADER]
station_id       = {station_id}
station_name     = {station_name}
longitude        = {longitude}
latitude         = {latitude}
altitude         = {altitude_meters}
nodata           = -999
tz               = {timezone}
source           = OpenMeteo
prototype        = SNOWPACK
ProfileDate      = {profile_date}
HS_Last          = 0.0000
SlopeAngle       = {slope_angle}
SlopeAzi         = {slope_azimuth}
nSoilLayerData   = 0
nSnowLayerData   = 0
SoilAlbedo       = 0.09
BareSoil_z0      = 0.020
CanopyHeight     = 0.00
CanopyLeafAreaIndex = 0.00
CanopyDirectThroughfall = 1.00
ErosionLevel     = 0
TimeCountDeltaHS = 0.000000
WindScalingFactor = 1.00

fields           = timestamp Layer_Thick  T  Vol_Frac_I  Vol_Frac_W  Vol_Frac_V  Vol_Frac_S Rho_S Conduc_S HeatCapac_S  rg  rb  dd  sp  mk mass_hoar ne CDot metamo
[DATA]
"""

    return content

def create_ini_content(station_id, filename, snowfiles, meas_tss, enforce_measured_snow_heights,
                        coord_sys, coord_param, timezone, write_profiles, write_timeseries, write_snowpack,
                        snow_erosion, snow_redistribution, hs_density_parameterization,
                        meteo_path="../input", output_path="./output", snow_output_path="./output",
                        ts_days_between=0.25, prof_days_between=1, snow_days_between=365, first_backup=365):
    """Create .ini file content that mirrors the master configuration structure."""

    def format_bool(value):
        if isinstance(value, str):
            return "TRUE" if value.strip().lower() == "true" else "FALSE"
        return "TRUE" if bool(value) else "FALSE"

    def format_option(value):
        return str(value).strip().upper()

    def format_number(value):
        return format(value, "g") if isinstance(value, (int, float)) else str(value)

    meteo_path = meteo_path or "../input"
    output_path = output_path or "./output"
    snow_output_path = snow_output_path or output_path

    lines = []
    lines.append("############################## INPUTS ##############################")
    lines.append("")
    lines.append("[INPUT]")
    lines.append(f"COORDSYS={coord_sys}")
    lines.append(f"COORDPARAM={coord_param}")
    lines.append(f"TIME_ZONE={timezone}")
    lines.append("METEO=SMET")
    lines.append("SNOW=SMET")
    lines.append("SLOPE_FROM_SNO=TRUE")
    lines.append(f"METEOPATH={meteo_path}")
    lines.append(f"METEOFILE1={filename}")
    for index, snowfile in enumerate(snowfiles, start=1):
        lines.append(f"SNOWFILE{index}={snowfile}")

    lines.append("")
    lines.append("[FILTERS]")
    lines.append("TA::FILTER1=ADD")
    lines.append("TA::ARG1::TYPE=CST")
    lines.append("TA::ARG1::CST=273.15")
    lines.append("RH::FILTER1=MULT")
    lines.append("RH::ARG1::TYPE=CST")
    lines.append("RH::ARG1::CST=0.01")

    lines.append("")
    lines.append("[INPUTEDITING]")
    lines.append("*::EDIT1=CREATE")
    lines.append("*::ARG1::PARAM=TSG")
    lines.append("*::ARG1::ALGORITHM=CST")
    lines.append("*::ARG1::VALUE=273.15")

    lines.append("")
    lines.append("[INTERPOLATIONS1D]")
    lines.append("MAX_GAP_SIZE=86400")
    lines.append("PSUM::RESAMPLE1=ACCUMULATE")
    lines.append("PSUM::ARG1::PERIOD=1800")
    lines.append("DW::RESAMPLE1=NEAREST")
    lines.append("ILWR::RESAMPLE1=LINEAR")
    lines.append("ISWR::RESAMPLE1=LINEAR")
    lines.append("RH::RESAMPLE1=LINEAR")
    lines.append("TA::RESAMPLE1=LINEAR")
    lines.append("VW::RESAMPLE1=LINEAR")

    lines.append("")
    lines.append("############################## MODEL ##############################")
    lines.append("")
    lines.append("[SNOWPACK]")
    lines.append("CALCULATION_STEP_LENGTH=30")
    lines.append("ROUGHNESS_LENGTH=0.002")
    lines.append("HEIGHT_OF_METEO_VALUES=2")
    lines.append("HEIGHT_OF_WIND_VALUE=10")
    lines.append(f"ENFORCE_MEASURED_SNOW_HEIGHTS={format_bool(enforce_measured_snow_heights)}")
    lines.append("SW_MODE=INCOMING")
    lines.append("ATMOSPHERIC_STABILITY=NEUTRAL")
    lines.append("CHANGE_BC=FALSE")
    lines.append("SNP_SOIL=FALSE")
    lines.append("CANOPY=FALSE")
    lines.append(f"MEAS_TSS={format_bool(meas_tss)}")

    lines.append("")
    lines.append("[SNOWPACKADVANCED]")
    lines.append(f"NUMBER_SLOPES={len(snowfiles)}")
    lines.append(f"SNOW_REDISTRIBUTION={format_option(snow_redistribution)} # Specifies if redistribution of snow is allowed from windward (luv) to lee slopes. The snow erosion on virtual slopes will contribute to the drifting snow index (default: FALSE).")
    lines.append("THRESH_RAIN=1.4          # Increase default from 1.2 to 1.4 based on feedback from Sherbrooke and Zach Miller")
    lines.append("T_CRAZY_MIN=140")
    lines.append("T_CRAZY_MAX=360")
    lines.append("SSI_IS_RTA=TRUE         # Assme this is helpful for Flo's code?")
    lines.append(f"SNOW_EROSION={format_option(snow_erosion)}")
    lines.append(f"HS_DENSITY_PARAMETERIZATION={format_option(hs_density_parameterization)}")

    lines.append("")
    lines.append("############################## OUTPUTS ##############################")
    lines.append("")
    lines.append("[OUTPUT]")
    lines.append("## Smet settings")
    lines.append("METEO=SMET")
    lines.append("SMET_APPEND=TRUE")
    lines.append("SMET_OVERWRITE=FALSE")
    lines.append("SMET_PLOT_HEADERS=FALSE")
    lines.append(f"COORDSYS={coord_sys if coord_sys else 'PROJ4'}")
    lines.append(f"COORDPARAM={coord_param if coord_param else '4326'}")
    lines.append(f"TIME_ZONE={timezone}")

    lines.append("## Timeseries smet files")
    lines.append(f"TS_WRITE={format_bool(write_timeseries)}           # Writes to METEOPATH")
    lines.append("TS_FORMAT=SMET")
    lines.append("TS_START=0")
    lines.append(f"TS_DAYS_BETWEEN={format_number(ts_days_between)}    # 3 hours=.125, 1 hour=4.1666e-2, 0.5 hour=2.08333e-2, 0.25 hour=1.04167e-2")
    lines.append("AVGSUM_TIME_SERIES=FALSE # If AVGSUM_TIME_SERIES is set, mean fluxes and cumulated masses since last dump are written, else current energy fluxes, cumulated masses over last COMPUTATION_STEP_LENGTH (recommended setting in operational mode). This is forced to false when using virtual slopes!")
    lines.append("CUMSUM_MASS=FALSE       # If CUMSUM_MASS is set, current value of cumulated masses since begin of run are dumped. See PRECIP_RATES for precipitation. This is forced to false when using virtual slopes!")
    lines.append("PRECIP_RATES=FALSE      # Write precipitation as rates (kg m-2 h-1, default) or as sums over the output time step. Please note that the units label in the output file and in SN_GUI will remain kg m-2 h-1 and will therefore not match sums.")
    lines.append("OUT_HAZ=TRUE")
    lines.append("OUT_HEAT=FALSE")
    lines.append("OUT_LW=FALSE")
    lines.append("OUT_METEO=TRUE")
    lines.append("OUT_SW=FALSE")
    lines.append("OUT_STAB=FALSE")
    lines.append("OUT_T=FALSE")

    lines.append("## Pro files")
    lines.append(f"PROF_WRITE={format_bool(write_profiles)}         # Writes to METEOPATH")
    lines.append("PROF_FORMAT=PRO")
    lines.append("PROF_AGE_OR_DATE=DATE")
    lines.append(f"PROF_DAYS_BETWEEN={format_number(prof_days_between)}")

    lines.append("## Sno files")
    lines.append(f"SNOW_WRITE={format_bool(write_snowpack)}         # Writes to SNOWPATH")
    lines.append("SNOW=SMET")
    lines.append(f"SNOWPATH={snow_output_path}")
    lines.append(f"SNOW_DAYS_BETWEEN={format_number(snow_days_between)}   # Dump snow files every SNOW_DAYS_BETWEEN day (STATION_YYYYMMDDHHMMSS.sno)")
    lines.append(f"FIRST_BACKUP={format_number(first_backup)}        # First dump of sno files happens at end of run or after FIRST_BACKUP days, whichever happens first")

    lines.append("## Haz files")
    lines.append("HAZ_WRITE=FALSE         # Writes to SNOWPATH with same dump settings as SNOW")
    lines.append(f"METEOPATH={output_path}")

    return "\n".join(lines) + "\n"

def get_slope_filename(angle, azimuth, slope_index, include_flat):
    """Get filename for slope naming convention.

    Naming rules:
    - Flat slope (angle == 0.0): Returns 'keystone' (no number)
    - Non-flat slopes: Returns 'keystone1', 'keystone2', etc.
    - Numbering depends on whether flat slope is included:
      * If flat included: non-flat slopes start from index 1
      * If flat not included: all slopes start from index 1

    Args:
        angle: Slope angle in degrees (0.0 for flat)
        azimuth: Slope azimuth in degrees
        slope_index: Index of slope in the slopes list
        include_flat: Whether flat slope is included in the list

    Returns:
        Filename base (without .sno extension)
    """
    if angle == 0.0:
        return "keystone"  # Flat slope gets no number
    else:
        # For numbered slopes, determine the number based on position
        if include_flat:
            # If flat is included, non-flat slopes start from index 1
            return f"keystone{slope_index}"
        else:
            # If no flat, start numbering from 1
            return f"keystone{slope_index + 1}"

def main():
    """Main execution function"""

    print("SNOWPACK Configuration Files Generation")
    print("=" * 50)
    print(f"Station: {station_name} ({station_id})")
    print(f"Location: {latitude:.6f}°N, {longitude:.6f}°W, {altitude_meters}m")
    print(f"Sno Directory: {sno_directory}")
    print(f"Ini Directory: {ini_directory}")
    print("=" * 50)

    if not generate_config_files:
        print("Configuration generation disabled. Set 'generate_config_files' to True to generate files.")
        return

    # Validate profile_date format
    try:
        from datetime import datetime
        datetime.strptime(profile_date, "%Y-%m-%dT%H:%M:%S")
    except ValueError:
        print(f"Warning: profile_date format may be incorrect. Expected format: YYYY-MM-DDTHH:MM:SS")
        print(f"  Current value: {profile_date}")

    # Validate directories
    try:
        os.makedirs(sno_directory, exist_ok=True)
        os.makedirs(ini_directory, exist_ok=True)
    except Exception as e:
        print(f"Error creating directories: {e}")
        return

    output_directory_path = os.path.join(ini_directory, "output")
    os.makedirs(output_directory_path, exist_ok=True)

    try:
        meteo_path_relative = os.path.relpath(sno_directory, ini_directory)
    except ValueError:
        meteo_path_relative = sno_directory

    output_path_relative = "./output"
    snow_output_relative = "./output"

    # Generate slopes
    slopes = generate_slopes(include_flat, north_slope, east_slope, south_slope, west_slope, custom_directions, default_slope_angle)

    # Limit to requested number of slopes
    if len(slopes) > num_slopes:
        slopes = slopes[:num_slopes]
        print(f"Warning: Limited to {num_slopes} slopes as requested")

    # Create multiple .sno files and collect snowfile names
    snowfiles = []
    sno_files_created = []

    for i, (angle, azimuth) in enumerate(slopes):
        # Get filename based on new naming convention
        sno_filename = get_slope_filename(angle, azimuth, i, include_flat) + ".sno"
        snowfiles.append(sno_filename)

        # Create .sno content for this slope
        sno_content = create_sno_content(station_id, station_name, longitude, latitude, altitude_meters, timezone, profile_date, angle, azimuth)

        # Write .sno file to input directory
        sno_filepath = os.path.join(sno_directory, sno_filename)
        try:
            with open(sno_filepath, "w") as f:
                f.write(sno_content)
            sno_files_created.append(sno_filepath)
        except Exception as e:
            print(f"Error writing {sno_filepath}: {e}")
            continue

    # Create .ini content with multiple SNOWFILE entries
    ini_content = create_ini_content(
        station_id,
        f"{station_id}.smet",
        snowfiles,
        meas_tss,
        enforce_measured_snow_heights,
        coord_sys,
        coord_param,
        timezone,
        write_profiles,
        write_timeseries,
        write_snowpack,
        snow_erosion,
        snow_redistribution,
        hs_density_parameterization,
        meteo_path=meteo_path_relative,
        output_path=output_path_relative,
        snow_output_path=snow_output_relative
    )

    # Write .ini file to keystone directory
    ini_filename = os.path.join(ini_directory, f"{station_id}.ini")
    try:
        with open(ini_filename, "w") as f:
            f.write(ini_content)
    except Exception as e:
        print(f"Error writing {ini_filename}: {e}")
        return

    # Display results
    print(f"\n Configuration files created:")
    print(f"   .ini file: {ini_filename}")
    print(f"   .sno files:")
    for sno_file in sno_files_created:
        print(f"     {sno_file}")

    print(f"\nVirtual slopes configured: {len(slopes)}")
    for i, (angle, azimuth) in enumerate(slopes):
        sno_filename = get_slope_filename(angle, azimuth, i, include_flat)
        if angle == 0.0:
            print(f"   Slope {i+1}: Flat (0°) -> {sno_filename}.sno")
        else:
            direction = ""
            if azimuth == 0.0:
                direction = "North"
            elif azimuth == 90.0:
                direction = "East"
            elif azimuth == 180.0:
                direction = "South"
            elif azimuth == 270.0:
                direction = "West"
            else:
                direction = f"{azimuth}°"
            print(f"   Slope {i+1}: {angle}° slope facing {direction} -> {sno_filename}.sno")

    print(f"\nSNOWPACK settings:")
    print(f"   MEAS_TSS = {meas_tss}")
    print(f"   ENFORCE_MEASURED_SNOW_HEIGHTS = {enforce_measured_snow_heights}")
    print(f"   NUMBER_SLOPES = {len(slopes)}")
    print(f"   SNOW_EROSION = {snow_erosion}")
    print(f"   SNOW_REDISTRIBUTION = {snow_redistribution}")
    print(f"   HS_DENSITY_PARAMETERIZATION = {hs_density_parameterization}")

    print("\n🎉 SNOWPACK configuration generation complete!")

# Run the main function
if __name__ == "__main__":
    main()

SNOWPACK Configuration Files Generation
Station: watrous_E_NTL (watrous)
Location: 39.714380°N, -105.844750°W, 3596.6400000000003m
Sno Directory: /content/input
Ini Directory: /content/config

 Configuration files created:
   .ini file: /content/config/watrous.ini
   .sno files:
     /content/input/keystone.sno
     /content/input/keystone1.sno
     /content/input/keystone2.sno
     /content/input/keystone3.sno
     /content/input/keystone4.sno

Virtual slopes configured: 5
   Slope 1: Flat (0°) -> keystone.sno
   Slope 2: 38.0° slope facing North -> keystone1.sno
   Slope 3: 38.0° slope facing East -> keystone2.sno
   Slope 4: 38.0° slope facing South -> keystone3.sno
   Slope 5: 38.0° slope facing West -> keystone4.sno

SNOWPACK settings:
   MEAS_TSS = false
   ENFORCE_MEASURED_SNOW_HEIGHTS = false
   NUMBER_SLOPES = 5

🎉 SNOWPACK configuration generation complete!


### Master INI defaults

The configuration files generated below now copy the structure and defaults from `master.ini`. You can still change site-specific details (location, time zone, slope files, relative paths) through the form inputs in the cell above. If you need to tweak any of the master defaults, adjust the parameters passed into `create_ini_content` where noted in the code comments.


In [ ]:
# @title SMET Handling, API Setup, and Parameters

# @markdown ### Location Settings are used from above.
# Pulling from the configuration cell
# latitude = 39.56858687967004  # @param {type:"number"}
# longitude = -105.91900397453021  # @param {type:"number"}
# altitude_input = 3614  # @param {type:"number"}
# altitude_unit = "meters"  # @param ["meters", "feet"]

# @markdown ### Station Information is used from above.
# Pulling from the configuration cell
# station_name = "keystone_model"  # @param {type:"string"}

# @markdown ### Time Period
start_date = "2024-11-01"  # @param {type:"date"}
end_date = "2025-04-30"    # @param {type:"date"}

# @markdown ### Weather Model
model_selection = "ifs"  # @param ["nbm", "ifs", "gfs", "gfs_hrrr"]

# @markdown ### HS (Snow Depth) Source
hs_source = "model"  # @param ["model", "snodas"]

# @markdown ### Open-Meteo Elevation
openmeteo_elevation_mode = "use_selected_altitude"  # @param ["use_model_elevation", "use_selected_altitude"]

# @markdown ### SMET Generation
generate_files = True  # @param {type:"boolean"}

# --- Helper functions ----------------------------------------------------

# Import additional modules needed for SNODAS
import struct
import tarfile
import gzip
import urllib.request
import urllib.error
from io import BytesIO


def get_snodas_snow_depth(lat, lon, date_str, cache_dir="snodas_cache", debug=False):
    """
    Download and extract SNODAS snow depth from NSIDC.
    
    SNODAS Units:
    - Raw data: millimeters (mm) stored as integer (snow_depth_raw / 1000.0)
    - Returns: meters (m) as float
    
    This is different from Open-Meteo which returns centimeters (cm).
    """
    SNODAS_NODATA = -9999
    
    # Grid configurations (detected from file size)
    GRID_CONFIGS = {
        'old': {'XMIN': -124.73375000000000, 'YMAX': 52.87458333333333, 
                'XMAX': -66.94208333333333, 'YMIN': 24.94958333333333,
                'NCOLS': 6935, 'NROWS': 3351, 'name': 'Pre-Oct-2013'},
        'new': {'XMIN': -124.73333333333333, 'YMAX': 52.87500000000000,
                'XMAX': -66.94166666666667, 'YMIN': 24.95000000000000,
                'NCOLS': 3353, 'NROWS': 3353, 'name': 'Post-Oct-2013'}
    }
    
    # Check location bounds
    if lat < 24.95 or lat > 52.88 or lon < -124.74 or lon > -66.94:
        return None
    
    # Construct URL
    tar_filename = f"SNODAS_{date_str}.tar"
    data_base = "https://noaadata.apps.nsidc.org/NOAA/G02158/masked"
    year = date_str[:4]
    month = date_str[4:6]
    month_names = ["01_Jan", "02_Feb", "03_Mar", "04_Apr", "05_May", "06_Jun",
                   "07_Jul", "08_Aug", "09_Sep", "10_Oct", "11_Nov", "12_Dec"]
    month_dir = month_names[int(month) - 1]
    data_url = f"{data_base}/{year}/{month_dir}/{tar_filename}"
    
    os.makedirs(cache_dir, exist_ok=True)
    cache_path = os.path.join(cache_dir, tar_filename)
    
    try:
        # Download or use cache
        if os.path.exists(cache_path):
            with open(cache_path, 'rb') as f:
                tar_data = BytesIO(f.read())
        else:
            if debug:
                print(f"  Downloading {date_str}...")
            with urllib.request.urlopen(data_url, timeout=60) as response:
                tar_data = BytesIO(response.read())
                with open(cache_path, 'wb') as f:
                    f.write(tar_data.getvalue())
            tar_data.seek(0)
        
        # Extract and decompress
        with tarfile.open(fileobj=tar_data, mode='r') as tar:
            for member in tar.getmembers():
                if '1036' in member.name and member.name.endswith('.dat.gz'):
                    snow_depth_gz_file = tar.extractfile(member)
                    break
            else:
                return None
        
        with gzip.open(snow_depth_gz_file, 'rb') as gz_file:
            data = gz_file.read()
        
        # Detect grid from file size
        num_values = len(data) // 2
        grid_config = None
        for config in GRID_CONFIGS.values():
            if num_values == config['NCOLS'] * config['NROWS']:
                grid_config = config
                break
        
        if grid_config is None:
            return None
        
        # Parse binary data
        SNODAS_NCOLS = grid_config['NCOLS']
        SNODAS_NROWS = grid_config['NROWS']
        values = struct.unpack(f">{SNODAS_NCOLS * SNODAS_NROWS}h", data)
        snow_depth_array = np.array(values).reshape((SNODAS_NROWS, SNODAS_NCOLS))
        
        # Calculate grid coordinates
        SNODAS_XMIN = grid_config['XMIN']
        SNODAS_YMAX = grid_config['YMAX']
        SNODAS_CELLSIZE_X = (grid_config['XMAX'] - SNODAS_XMIN) / SNODAS_NCOLS
        SNODAS_CELLSIZE_Y = (SNODAS_YMAX - grid_config['YMIN']) / SNODAS_NROWS
        
        col = int((lon - SNODAS_XMIN) / SNODAS_CELLSIZE_X)
        row = int((SNODAS_YMAX - lat) / SNODAS_CELLSIZE_Y)
        col = max(0, min(SNODAS_NCOLS - 1, col))
        row = max(0, min(SNODAS_NROWS - 1, row))
        
        # Extract value
        snow_depth_raw = snow_depth_array[row, col]
        if snow_depth_raw == SNODAS_NODATA or snow_depth_raw < 0:
            return None
        
        snow_depth_m = snow_depth_raw / 1000.0
        return snow_depth_m if snow_depth_m >= 0.01 else 0.0
            
    except Exception as e:
        if debug:
            print(f"  Error: {e}")
        return None


def resolve_openmeteo_elevation(mode: str, station_alt_m: float) -> Optional[float]:
    """
    Decide which elevation to send to the Open-Meteo API.
    - use_model_elevation     -> None (API uses its DEM; statistical downscaling stays enabled)
    - use_selected_altitude   -> the altitude value converted to meters above
    """
    mode = mode.lower()
    if mode == "use_model_elevation":
        return None
    if mode == "use_selected_altitude":
        return float(station_alt_m)
    raise ValueError(f"Unknown elevation mode: {mode}")


def create_smet_from_weather_data(weather_df: pd.DataFrame,
                                  output_path: str,
                                  station_id: str,
                                  station_name: str,
                                  latitude: float,
                                  longitude: float,
                                  altitude_meters: float,
                                  timezone: float = -7) -> None:
    """
    Create a SMET file from a weather DataFrame and write it to disk.
    
    Expected DataFrame Units (SMET format):
    - TA: Kelvin (K)
    - RH: Fraction (0-1)
    - TSG: Kelvin (K) - ground temperature
    - VW: m/s
    - DW: Degrees (0-360)
    - HS: Meters (m) - snow depth
    - ISWR: W/m²
    - PSUM: mm

    Args:
        weather_df: DataFrame containing weather data with timestamp column
                   Must have units matching SMET format (see above)
        output_path: Path where SMET file will be written
        station_id: Station identifier
        station_name: Station name
        latitude: Latitude in degrees
        longitude: Longitude in degrees
        altitude_meters: Altitude in meters
        timezone: Timezone offset (default: -7 for Mountain Time)
    """
    if "timestamp" in weather_df.columns:
        weather_df = weather_df.copy()
        weather_df["timestamp"] = pd.to_datetime(weather_df["timestamp"])
    else:
        weather_df = weather_df.copy()
        weather_df.reset_index(inplace=True)
        weather_df["timestamp"] = pd.to_datetime(weather_df["timestamp"])

    fields = ["timestamp", "TA", "RH", "TSG", "VW", "DW", "ISWR", "PSUM"]
    if "HS" in weather_df.columns:
        fields.append("HS")

    output_dir = os.path.dirname(output_path)
    if output_dir:
        os.makedirs(output_dir, exist_ok=True)

    with open(output_path, "w") as f:
        f.write("SMET 1.1 ASCII\n")
        f.write("[HEADER]\n")
        f.write(f"station_id = {station_id}\n")
        f.write(f"station_name = {station_name}\n")
        f.write(f"latitude = {latitude:.10f}\n")
        f.write(f"longitude = {longitude:.10f}\n")
        f.write(f"altitude = {altitude_meters}\n")
        f.write("nodata = -777\n")
        f.write(f"Tz = {timezone}\n")
        f.write(f"fields = {'\t'.join(fields)}\n")

        field_count = len(fields)
        units_offset = ["0"] * field_count
        units_multiplier = ["1"] * field_count
        f.write(f"units_offset     = {' '.join(units_offset)}\n")
        f.write(f"units_multiplier = {' '.join(units_multiplier)}\n")

        f.write("[DATA]\n")
        for _, row in weather_df.iterrows():
            values = []
            for field in fields:
                if field == "timestamp":
                    values.append(row[field].strftime("%Y-%m-%dT%H:%M:%S"))
                else:
                    val = row.get(field, -777)
                    if pd.isna(val):
                        val = -777
                    values.append(f"{val:.2f}")
            f.write("\t".join(values) + "\n")


def fetch_openmeteo_historical(lat: float,
                               lon: float,
                               start_date: str,
                               end_date: str,
                               model: str = "gfs_global",
                               elevation: Optional[float] = None) -> Optional[pd.DataFrame]:
    """
    Fetch historical forecast data from the Open-Meteo API.
    
    Open-Meteo API Units (as returned):
    - temperature_2m: Celsius (°C)
    - relative_humidity_2m: Percentage (0-100)
    - wind_speed_10m: m/s (specified via wind_speed_unit parameter)
    - wind_direction_10m: Degrees (0-360)
    - snow_depth: Centimeters (cm)
    - direct_radiation: W/m²
    - precipitation: Millimeters (mm)
    
    Returns DataFrame with SMET-compatible units:
    - TA: Kelvin (K) - converted from Celsius
    - RH: Fraction (0-1) - converted from percentage
    - VW: m/s - already correct
    - DW: Degrees - already correct
    - HS: Centimeters (cm) - will be converted to meters later if using model data
    - ISWR: W/m² - already correct
    - PSUM: mm - already correct
    """
    cache_session = requests_cache.CachedSession(".cache", expire_after=3600)
    retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
    openmeteo = openmeteo_requests.Client(session=retry_session)

    url = "https://historical-forecast-api.open-meteo.com/v1/forecast"
    params = {
        "latitude": lat,
        "longitude": lon,
        "start_date": start_date,
        "end_date": end_date,
        "hourly": [
            "temperature_2m",
            "relative_humidity_2m",
            "wind_speed_10m",
            "wind_direction_10m",
            "snow_depth",
            "direct_radiation",
            "precipitation",
        ],
        "models": model,
        "wind_speed_unit": "ms",
    }

    if elevation is not None:
        params["elevation"] = elevation

    print(f"Fetching {model} data for {lat}, {lon} from {start_date} to {end_date}")
    if "elevation" in params:
        print(f"Open-Meteo elevation parameter: {params['elevation']} (m)")

    try:
        responses = openmeteo.weather_api(url, params=params)
        response = responses[0]

        print(f"Coordinates: {response.Latitude()}°N {response.Longitude()}°E")
        print(f"Elevation: {response.Elevation()} m asl")

        hourly = response.Hourly()
        hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
        hourly_relative_humidity_2m = hourly.Variables(1).ValuesAsNumpy()
        hourly_wind_speed_10m = hourly.Variables(2).ValuesAsNumpy()
        hourly_wind_direction_10m = hourly.Variables(3).ValuesAsNumpy()
        hourly_snow_depth = hourly.Variables(4).ValuesAsNumpy()
        hourly_direct_radiation = hourly.Variables(5).ValuesAsNumpy()
        hourly_precipitation = hourly.Variables(6).ValuesAsNumpy()

        time_index = pd.date_range(
            start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
            end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
            freq=pd.Timedelta(seconds=hourly.Interval()),
            inclusive="left",
        )

        # Convert Open-Meteo units to SMET-compatible units
        # TA: Open-Meteo returns Celsius, SMET requires Kelvin
        ta_kelvin = hourly_temperature_2m + 273.15
        
        # RH: Open-Meteo returns percentage (0-100), SMET requires fraction (0-1)
        rh_fraction = hourly_relative_humidity_2m / 100.0
        
        # VW: Open-Meteo returns m/s (via wind_speed_unit="ms"), SMET requires m/s - already correct
        vw_ms = hourly_wind_speed_10m
        
        # DW: Open-Meteo returns degrees, SMET requires degrees - already correct
        dw_degrees = hourly_wind_direction_10m
        
        # HS: Open-Meteo returns centimeters (cm), will convert to meters later if using model data
        # SNODAS returns meters directly, so we keep Open-Meteo in cm for now
        hs_cm = hourly_snow_depth
        
        # ISWR: Open-Meteo returns W/m², SMET requires W/m² - already correct
        iswr_wm2 = hourly_direct_radiation
        
        # PSUM: Open-Meteo returns mm, SMET requires mm - already correct
        psum_mm = hourly_precipitation
        
        df = pd.DataFrame(
            {
                "timestamp": time_index,
                "TA": ta_kelvin,           # Units: Kelvin (K)
                "RH": rh_fraction,         # Units: Fraction (0-1)
                "VW": vw_ms,               # Units: m/s
                "DW": dw_degrees,          # Units: Degrees
                "HS": hs_cm,               # Units: Centimeters (cm) - will convert to meters later
                "ISWR": iswr_wm2,          # Units: W/m²
                "PSUM": psum_mm,           # Units: mm
            }
        )
        
        # TSG (ground temperature) is not available from Open-Meteo API
        # Using assumed value of 273.15 K (0°C) as a reasonable default
        # SNOWPACK will calculate actual ground temperature during simulation
        df["TSG"] = 273.15  # Units: Kelvin (K)
        
        # Validate temperature range (reasonable bounds: 200-330 K = -73°C to 57°C)
        # Flag extreme values but don't fail - SNOWPACK will handle validation
        invalid_ta = (df["TA"] < 200.0) | (df["TA"] > 330.0)
        if invalid_ta.any():
            invalid_count = invalid_ta.sum()
            print(f"⚠ Warning: {invalid_count} temperature values outside reasonable range (200-330 K)")
            print(f"  Min TA: {df['TA'].min():.2f} K, Max TA: {df['TA'].max():.2f} K")
            print(f"  These values may cause SNOWPACK to fail. Check input data quality.")
        
        # Validate relative humidity range (should be 0-1)
        invalid_rh = (df["RH"] < 0.0) | (df["RH"] > 1.0)
        if invalid_rh.any():
            invalid_count = invalid_rh.sum()
            print(f"⚠ Warning: {invalid_count} relative humidity values outside range (0-1)")
            print(f"  Min RH: {df['RH'].min():.4f}, Max RH: {df['RH'].max():.4f}")
            # Clamp to valid range
            df.loc[df["RH"] < 0.0, "RH"] = 0.0
            df.loc[df["RH"] > 1.0, "RH"] = 1.0
        
        df = df.replace([np.inf, -np.inf], np.nan)
        
        # Check for NaN values and report
        nan_counts = df.isna().sum()
        if nan_counts.any():
            print(f"⚠ Warning: Found NaN values in data:")
            for col, count in nan_counts.items():
                if count > 0:
                    print(f"  {col}: {count} NaN values")
            # Replace NaN with nodata values for SMET compatibility
            df = df.fillna(-777)
            print("  Replaced NaN values with nodata (-777)")
        
        print(f"Fetched {len(df)} records")
        return df

    except Exception as e:
        print(f"Error fetching data: {e}")
        return None


# --- Derived values / summary -------------------------------------------

# Validate dates
try:
    from datetime import datetime
    start_dt = datetime.strptime(start_date, "%Y-%m-%d")
    end_dt = datetime.strptime(end_date, "%Y-%m-%d")
    if start_dt >= end_dt:
        raise ValueError(f"start_date ({start_date}) must be before end_date ({end_date})")
except ValueError as e:
    print(f"Date validation error: {e}")
    print("Please check your date inputs and try again.")
    raise

# Validate that altitude_meters is available (from configuration cell)
try:
    altitude_meters
except NameError:
    print("Error: altitude_meters not found. Please run the configuration cell first.")
    raise

# altitude_meters = altitude # Commented out direct assignment
openmeteo_elevation = resolve_openmeteo_elevation(openmeteo_elevation_mode, altitude_meters)

print("SMET Generation Parameters")
print("=" * 50)
print(f"Location: {latitude}, {longitude}")
print(f"Altitude input: {altitude} {altitude_unit} ({altitude_meters:.2f} m used for calculations)")
# print(f"Altitude: {altitude_meters:.2f} m")
print(f"Station: {station_name}")
print(f"Period: {start_date} to {end_date}")
print(f"Model: {model_selection}")
print(f"HS Source: {hs_source}")
print(f"Generate: {generate_files}")
if openmeteo_elevation is None:
    print("Open-Meteo elevation: model elevation (automatic downscaling)")
else:
    print(f"Open-Meteo elevation: {openmeteo_elevation:.2f} m (selected altitude)")
print("=" * 50)

SMET Generation Parameters
Location: 39.71438, -105.84475
Altitude input: 11800 feet (3596.64 m used for calculations)
Station: watrous_E_NTL
Period: 2024-11-01 to 2025-04-30
Model: ifs
Generate: True
Open-Meteo elevation: 3596.64 m (selected altitude)


In [ ]:
# @title Generate SMET Files
# @markdown Run this cell to generate SMET files with the parameters above

# Import for parallel processing of cached SNODAS data
from concurrent.futures import ThreadPoolExecutor, as_completed

# Hardcoded output path
# output_directory = "/content/input" # Commented out hardcoded path

# Use sno_directory from the configuration cell
output_directory = sno_directory  # sno directory is input folder

# Model mapping
model_mapping = {
    "nbm": "ncep_nbm_conus",
    "ifs": "ecmwf_ifs",
    "gfs": "gfs_global",
    "gfs_hrrr": "gfs_hrrr"
}


if generate_files:
    print("Starting SMET generation...")

    # Parse model selection
    selected_models = [model_selection]

    # Validate and map models
    valid_models = ['nbm', 'ifs', 'gfs', 'gfs_hrrr']
    selected_models = [model for model in selected_models if model in valid_models]

    if not selected_models:
        print("No valid models selected!")
        print(f"Available models: {', '.join(valid_models)}")
    else:
        print(f"Processing {len(selected_models)} model(s): {', '.join(selected_models)}")
        print("="*50)

        successful_files = []

        for model in selected_models:
            print(f"\nProcessing {model}...")

            # Map to Open-Meteo model name
            openmeteo_model = model_mapping[model]

            # Fetch data
            df = fetch_openmeteo_historical(
                lat=latitude,
                lon=longitude,
                start_date=start_date,
                end_date=end_date,
                model=openmeteo_model,
                elevation=openmeteo_elevation
            )

            if df is None or df.empty:
                print(f"Failed to fetch data for {model}")
                continue

            # Initialize SNODAS fallback flag (used later in unit conversion)
            snodas_fallback_used = False

            # Process SNODAS snow depth data if requested
            if hs_source == "snodas":
                print(f"\nProcessing SNODAS snow depth data...")
                
                # SNODAS coverage bounds (contiguous US)
                SNODAS_YMIN = 24.95
                SNODAS_YMAX = 52.83
                SNODAS_XMIN = -124.73
                SNODAS_XMAX = -66.95
                
                # Check if location is within SNODAS coverage
                if latitude < SNODAS_YMIN or latitude > SNODAS_YMAX or longitude < SNODAS_XMIN or longitude > SNODAS_XMAX:
                    print(f"⚠ Location ({latitude}°N, {longitude}°E) is outside SNODAS coverage area.")
                    print("SNODAS data is only available for the contiguous United States.")
                    print("All HS values set to nodata (-777). SNOWPACK will interpolate these values during model run.")
                    # Set all HS values to nodata
                    df['HS'] = -777
                else:
                    # Generate date range for SNODAS downloads
                    start_dt = pd.to_datetime(start_date)
                    end_dt = pd.to_datetime(end_date)
                    
                    # Create daily SNODAS values dictionary
                    snodas_snow_depth = {}
                    missing_dates = []
                    
                    # Generate all dates in range
                    all_dates = []
                    current_date = start_dt
                    while current_date <= end_dt:
                        all_dates.append(current_date)
                        current_date += timedelta(days=1)
                    
                    total_dates = len(all_dates)
                    print(f"Processing SNODAS data for {total_dates} days...")
                    print(f"Location: {latitude}°N, {longitude}°E")
                    
                    # Pre-check cache status
                    cache_dir = "snodas_cache"
                    cached_dates = []
                    uncached_dates = []
                    for current_date in all_dates:
                        date_str = current_date.strftime("%Y%m%d")
                        tar_filename = f"SNODAS_{date_str}.tar"
                        cache_path = os.path.join(cache_dir, tar_filename)
                        if os.path.exists(cache_path):
                            cached_dates.append(date_str)
                        else:
                            uncached_dates.append(date_str)
                    
                    print(f"  Cache status: {len(cached_dates)} cached, {len(uncached_dates)} need download")
                    
                    # Process cached dates in parallel (faster)
                    if cached_dates:
                        print(f"\nProcessing {len(cached_dates)} cached dates in parallel...")
                        with ThreadPoolExecutor(max_workers=4) as executor:
                            future_to_date = {
                                executor.submit(get_snodas_snow_depth, latitude, longitude, date_str, debug=False): date_str
                                for date_str in cached_dates
                            }
                            for future in as_completed(future_to_date):
                                date_str = future_to_date[future]
                                try:
                                    snow_depth = future.result()
                                    if snow_depth is not None:
                                        snodas_snow_depth[date_str] = snow_depth
                                        print(f"✓ {date_str}: {snow_depth:.3f} m (cached)")
                                    else:
                                        missing_dates.append(date_str)
                                        print(f"✗ {date_str}: No data (cached)")
                                except Exception as e:
                                    missing_dates.append(date_str)
                                    print(f"✗ {date_str}: Error (cached)")
                    
                    # Download uncached dates sequentially (to avoid overwhelming server)
                    if uncached_dates:
                        print(f"\nDownloading {len(uncached_dates)} new dates...")
                        for idx, date_str in enumerate(uncached_dates, 1):
                            progress = (idx / len(uncached_dates)) * 100
                            print(f"[{idx:3d}/{len(uncached_dates)}] ({progress:5.1f}%) {date_str}... ", end="", flush=True)
                            
                            try:
                                snow_depth = get_snodas_snow_depth(latitude, longitude, date_str, debug=False)
                                if snow_depth is not None:
                                    snodas_snow_depth[date_str] = snow_depth
                                    print(f"✓ {snow_depth:.3f} m")
                                else:
                                    missing_dates.append(date_str)
                                    print("✗ No data")
                            except Exception as e:
                                missing_dates.append(date_str)
                                print("✗ Error")
                    
                    if len(snodas_snow_depth) == 0:
                        print("\n⚠ No SNODAS data available for this date range.")
                        print("All HS values set to nodata (-777). SNOWPACK will interpolate these values during model run.")
                        df['HS'] = -777
                    else:
                        print(f"\n✓ Retrieved SNODAS snow depth for {len(snodas_snow_depth)}/{total_dates} days")
                        if len(missing_dates) > 0:
                            print(f"⚠ SNODAS data unavailable for {len(missing_dates)} dates - using nodata (-777).")
                            print("SNOWPACK will interpolate these gaps during model run.")
                        
                        # Replace model HS values with SNODAS values
                        # IMPORTANT UNIT CONVERSION:
                        # - df['HS'] is currently in cm (from Open-Meteo)
                        # - snodas_snow_depth[date_str] is in meters (from SNODAS function)
                        # - We replace cm values with meter values directly
                        # - SNODAS provides daily snapshots, apply same value to all hours of each day
                        replaced_count = 0
                        nodata_count = 0
                        
                        # Ensure HS column is float64 to avoid dtype warnings
                        if df['HS'].dtype != 'float64':
                            df['HS'] = df['HS'].astype('float64')
                        
                        for idx, row in df.iterrows():
                            date_str = row['timestamp'].strftime("%Y%m%d")
                            if date_str in snodas_snow_depth:
                                # Replace Open-Meteo HS (cm) with SNODAS HS (m)
                                # snodas_snow_depth[date_str] is already in meters
                                df.at[idx, 'HS'] = float(snodas_snow_depth[date_str])  # Units: meters (m)
                                replaced_count += 1
                            else:
                                # Set to nodata for missing dates
                                df.at[idx, 'HS'] = -777.0  # Nodata value (float)
                                nodata_count += 1
                        
                        print(f"✓ Applied SNODAS values to {replaced_count} hourly records")
                        if nodata_count > 0:
                            print(f"✓ Set {nodata_count} hourly records to nodata (-777) for missing dates")
            
            # ====================================================================
            # UNIT CONVERSION: Ensure all variables are in SMET-compatible units
            # ====================================================================
            # SMET Format Requirements:
            # - TA: Kelvin (K) ✓ (already converted from Celsius in fetch function)
            # - RH: Fraction (0-1) ✓ (already converted from percentage in fetch function)
            # - VW: m/s ✓ (already correct from Open-Meteo)
            # - DW: Degrees ✓ (already correct from Open-Meteo)
            # - HS: Meters (m) - NEEDS CONVERSION
            # - ISWR: W/m² ✓ (already correct from Open-Meteo)
            # - PSUM: mm ✓ (already correct from Open-Meteo)
            # - TSG: Kelvin (K) ✓ (already set to 273.15 K)
            # ====================================================================
            
            # HS (Snow Depth) Unit Conversion:
            # Open-Meteo returns: centimeters (cm)
            # SNODAS returns: meters (m)
            # SMET requires: meters (m)
            
            if hs_source == "model" and 'HS' in df.columns:
                # Using model data: Open-Meteo returns cm, convert to meters for SMET
                # df['HS'] is currently in cm from Open-Meteo
                df['HS'] = df['HS'] / 100.0  # Convert cm → m
                print(f"\n✓ Converted HS from cm to meters (model data)")
                print(f"  HS range: {df[df['HS'] >= 0]['HS'].min():.3f} - {df[df['HS'] >= 0]['HS'].max():.3f} m")
            elif hs_source == "snodas" and 'HS' in df.columns:
                # Check if fallback was used
                if snodas_fallback_used:
                    # Model data was used (fallback), convert from cm to meters
                    # df['HS'] is currently in cm from Open-Meteo
                    df['HS'] = df['HS'] / 100.0  # Convert cm → m
                    print("\n⚠ ALERT: SNODAS was selected but model fallback was used for HS values.")
                    print(f"  Converted HS from cm to meters (model fallback)")
                    print(f"  HS range: {df[df['HS'] >= 0]['HS'].min():.3f} - {df[df['HS'] >= 0]['HS'].max():.3f} m")
                else:
                    # SNODAS values are already in meters, nodata (-777) values are already set correctly
                    # df['HS'] contains SNODAS values in meters where available, -777 for missing dates
                    print(f"\n✓ Using SNODAS HS values (already in meters)")
                    valid_hs = df[df['HS'] != -777]['HS']
                    if len(valid_hs) > 0:
                        print(f"  HS range: {valid_hs.min():.3f} - {valid_hs.max():.3f} m")
                    print(f"  Nodata records: {(df['HS'] == -777).sum()}")
            
            # Final validation: Ensure all units are correct before writing SMET
            print(f"\n📊 Final data summary (SMET units):")
            
            # Check for any problematic values that might cause SNOWPACK to fail
            issues_found = []
            
            # Temperature validation (SNOWPACK typically fails if outside 200-330 K)
            ta_min, ta_max = df['TA'].min(), df['TA'].max()
            print(f"  TA: {ta_min:.2f} - {ta_max:.2f} K")
            if ta_min < 200.0 or ta_max > 330.0:
                issues_found.append(f"Temperature out of bounds (200-330 K): {ta_min:.2f} - {ta_max:.2f} K")
            # Check for individual problematic values
            extreme_ta = df[(df['TA'] < 200.0) | (df['TA'] > 330.0)]
            if len(extreme_ta) > 0:
                issues_found.append(f"{len(extreme_ta)} temperature values outside 200-330 K range")
                print(f"    ⚠ Found {len(extreme_ta)} extreme temperature values")
            
            # RH validation
            rh_min, rh_max = df['RH'].min(), df['RH'].max()
            print(f"  RH: {rh_min:.3f} - {rh_max:.3f} (fraction)")
            if rh_min < 0.0 or rh_max > 1.0:
                issues_found.append(f"Relative humidity out of bounds (0-1): {rh_min:.3f} - {rh_max:.3f}")
            
            # VW validation
            vw_min, vw_max = df['VW'].min(), df['VW'].max()
            print(f"  VW: {vw_min:.2f} - {vw_max:.2f} m/s")
            if vw_min < 0.0:
                issues_found.append(f"Negative wind speed values found")
            
            # HS validation
            if 'HS' in df.columns:
                valid_hs = df[df['HS'] != -777]['HS']
                if len(valid_hs) > 0:
                    hs_min, hs_max = valid_hs.min(), valid_hs.max()
                    print(f"  HS: {hs_min:.3f} - {hs_max:.3f} m")
                    if hs_min < 0.0:
                        issues_found.append(f"Negative HS values found")
                print(f"  HS nodata: {(df['HS'] == -777).sum()} records")
            
            # ISWR validation
            iswr_min, iswr_max = df['ISWR'].min(), df['ISWR'].max()
            print(f"  ISWR: {iswr_min:.1f} - {iswr_max:.1f} W/m²")
            if iswr_min < 0.0:
                issues_found.append(f"Negative ISWR values found")
            
            # PSUM validation
            psum_min, psum_max = df['PSUM'].min(), df['PSUM'].max()
            print(f"  PSUM: {psum_min:.2f} - {psum_max:.2f} mm")
            if psum_min < 0.0:
                issues_found.append(f"Negative precipitation values found")
            
            # Report any issues
            if issues_found:
                print(f"\n⚠ Potential data quality issues detected:")
                for issue in issues_found:
                    print(f"  - {issue}")
                print(f"  These may cause SNOWPACK to fail. Review the data before proceeding.")
            else:
                print(f"\n✓ All data ranges appear valid for SNOWPACK")

            # Create output filename with simplified naming
            output_file = os.path.join(output_directory, f"{station_id}.smet")
            # could add in _{model} if wanted to specify

            try:
                # Generate SMET file
                create_smet_from_weather_data(
                    weather_df=df,
                    output_path=output_file,
                    station_id=f"{station_id}_{model}",
                    station_name=f"{station_name}_{model}",
                    latitude=latitude,
                    longitude=longitude,
                    altitude_meters=altitude_meters,
                    timezone=timezone
                )

                print(f"SMET file created: {output_file}")
                print(f"Records: {len(df)}")
                print(f"Period: {df['timestamp'].min()} to {df['timestamp'].max()}")

                successful_files.append(output_file)

            except Exception as e:
                print(f"Error creating SMET for {model}: {e}")
                continue

        print(f"\nSuccessfully generated {len(successful_files)} SMET files:")
        for file in successful_files:
            print(f"  {file}")

        if successful_files:
            print(f"\nFiles saved to: {output_directory}")
else:
    print("Generation disabled. Set 'generate_files' to True to generate SMET files.")

Starting SMET generation...
Processing 1 model(s): ifs

Processing ifs...
Fetching ecmwf_ifs data for 39.71438, -105.84475 from 2024-11-01 to 2025-04-30
Open-Meteo elevation parameter: 3596.6400000000003 (m)
Coordinates: 39.68365478515625°N -105.875°E
Elevation: 3596.639892578125 m asl
Fetched 4344 records
SMET file created: /content/input/watrous.smet
Records: 4344
Period: 2024-11-01 00:00:00+00:00 to 2025-04-30 23:00:00+00:00

Successfully generated 1 SMET files:
  /content/input/watrous.smet

Files saved to: /content/input


In [ ]:
# @title Run this cell to run SNOWPACK
import subprocess
import sys

# Construct the path to the .ini file using variables from the configuration cell
ini_filepath = os.path.join(ini_directory, f"{station_id}.ini")

# Find the snowpack executable path dynamically
try:
    result = subprocess.run(["which", "snowpack"], capture_output=True, text=True, check=True)
    snowpack_executable_path = result.stdout.strip()
except (subprocess.CalledProcessError, FileNotFoundError):
    snowpack_executable_path = "snowpack"  # Fallback to PATH lookup

# Verify .ini file exists
if not os.path.exists(ini_filepath):
    raise FileNotFoundError(f"Configuration file not found: {ini_filepath}")

# Run SNOWPACK with proper error handling and real-time output streaming
# In Jupyter notebooks, we need to ensure output streams properly
try:
    # Use subprocess.Popen with line-by-line streaming to ensure output appears in notebook
    import sys
    process = subprocess.Popen(
        [snowpack_executable_path, "-c", ini_filepath, "-e", snowpack_end_date],
        cwd=ini_directory,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        universal_newlines=True,
        bufsize=1  # Line buffered
    )
    
    # Stream output line by line to notebook
    for line in process.stdout:
        print(line, end='', flush=True)
        sys.stdout.flush()
    
    # Wait for process to complete and get return code
    return_code = process.wait()
    
    if return_code != 0:
        raise subprocess.CalledProcessError(return_code, snowpack_executable_path)
        
except subprocess.CalledProcessError as e:
    print(f"\n❌ Error running SNOWPACK (exit code {e.returncode})")
    print("Check the output above for detailed error messages.")
    print("\nCommon issues:")
    print("  - Temperature values out of bounds (check TA range in data summary)")
    print("  - Missing or invalid SMET file")
    print("  - Configuration file errors")
    raise
except FileNotFoundError:
    print(f"❌ SNOWPACK executable not found: {snowpack_executable_path}")
    print("Please ensure SNOWPACK is compiled and in PATH")
    raise

In [48]:
# @title Download .pro files and Open niViz
# @markdown # 5 files will be download.
# @markdown # 1 = N, 2=E, 3=S, 4=W,
import glob, shutil, os
from IPython.display import FileLink, Markdown

# Check if running in Colab
try:
    from google.colab import files
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

# Determine temp directory based on environment
if IN_COLAB:
    temp_dir = "/content"
else:
    # For local environments, use current directory or a temp directory
    temp_dir = os.getcwd()

# Locate all .pro files
pro_files = glob.glob(os.path.join(ini_directory, 'output', '*.pro'))

if pro_files:
    # Copy all files to a simple location
    downloaded_files = []
    for pro_file in pro_files:
        filename = os.path.basename(pro_file)
        temp_file_path = os.path.join(temp_dir, filename)
        shutil.copy(pro_file, temp_file_path)
        downloaded_files.append(filename)

    # Create a zip file with all .pro files
    import zipfile
    zip_filename = 'snowpack_profiles.zip'
    zip_path = os.path.join(temp_dir, zip_filename)
    with zipfile.ZipFile(zip_path, 'w') as zipf:
        for filename in downloaded_files:
            file_path = os.path.join(temp_dir, filename)
            zipf.write(file_path, filename)

    # Download the zip file (only in Colab)
    if IN_COLAB:
        files.download(zip_path)
    else:
        print(f"Zip file created at: {zip_path}")
        print(f"Files included: {', '.join(downloaded_files)}")

    # Show how to open in niViz
    display(Markdown(f"""
    ---
    {len(downloaded_files)} SNOWPACK profile files have been downloaded as **`{zip_filename}`**.

    Next Step — View in niViz
    1. Go to https://run.niviz.org
    2. Click "File" → "Open Profile" or drag any of the downloaded files into the screen
    3. Select any of the downloaded files:
    """))

    for filename in downloaded_files:
        display(Markdown(f"   - **`{filename}`**"))

else:
    display(Markdown("No .pro files found in the output directory."))

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


    ---
    10 SNOWPACK profile files have been downloaded as **`snowpack_profiles.zip`**.

    Next Step — View in niViz
    1. Go to https://run.niviz.org
    2. Click "File" → "Open Profile" or drag any of the downloaded files into the screen
    3. Select any of the downloaded files:
    

   - **`ksp_model_ifs4_res.pro`**

   - **`ksp_model_ifs1_res.pro`**

...

   - **`ksp_model_ifs2_res.pro`**

   - **`ksp_model_ifs_res.pro`**

   - **`watrous_ifs1_res.pro`**

   - **`watrous_ifs_res.pro`**

   - **`watrous_ifs4_res.pro`**

# This cell can erase input and ini files to start over.  

In [ ]:
# @title Clean up generated files
# @markdown Run this cell to remove generated .sno and .ini files.

run_cleanup = False #@param {type:"boolean"}

import os
import shutil

# Use variables from the configuration cell
sno_directory_to_clear = sno_directory
ini_directory_to_clear = ini_directory

def clear_directory(directory_path):
    """Removes all files and subdirectories within a given directory.

    Args:
        directory_path: Path to directory to clear

    Returns:
        tuple: (success: bool, files_deleted: int, errors: list)
    """
    if not os.path.exists(directory_path):
        print(f"Directory not found: {directory_path}")
        return False, 0, []

    if not os.path.isdir(directory_path):
        print(f"Error: {directory_path} is not a directory")
        return False, 0, []

    # Count files before deletion
    file_count = len([f for f in os.listdir(directory_path)
                      if os.path.isfile(os.path.join(directory_path, f))])

    print(f"Clearing contents of: {directory_path}")
    print(f"  Found {file_count} file(s) and {len([d for d in os.listdir(directory_path) if os.path.isdir(os.path.join(directory_path, d))])} subdirectory(ies)")

    files_deleted = 0
    errors = []

    for item in os.listdir(directory_path):
        item_path = os.path.join(directory_path, item)
        try:
            if os.path.isfile(item_path) or os.path.islink(item_path):
                os.unlink(item_path)
                files_deleted += 1
            elif os.path.isdir(item_path):
                shutil.rmtree(item_path)
                files_deleted += 1
        except Exception as e:
            error_msg = f"Failed to delete {item_path}: {e}"
            errors.append(error_msg)
            print(f"  Warning: {error_msg}")

    if errors:
        print(f"  Completed with {len(errors)} error(s)")
    else:
        print(f"  Successfully deleted {files_deleted} item(s)")

    return len(errors) == 0, files_deleted, errors

if run_cleanup:
    print("=" * 60)
    print("WARNING: This will delete all files in the following directories:")
    print(f"  - {sno_directory_to_clear}")
    print(f"  - {ini_directory_to_clear}")
    print("=" * 60)

    # Clear the input (sno) directory
    sno_success, sno_count, sno_errors = clear_directory(sno_directory_to_clear)

    # Clear the config (ini) directory
    ini_success, ini_count, ini_errors = clear_directory(ini_directory_to_clear)

    total_errors = len(sno_errors) + len(ini_errors)
    if total_errors == 0:
        print("\n✓ Cleanup complete successfully.")
    else:
        print(f"\n⚠ Cleanup completed with {total_errors} error(s).")
else:
    print("Cleanup is disabled. Check the 'Run cleanup' box to enable.")